Libraries importeren

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split

D:\Anaconda3\envs\SmartSystems\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Dataset importeren

In [2]:
athletes = pd.read_csv("datasets/athlete_events.csv")
df = pd.DataFrame(athletes)

Overzicht dataset laten zien

In [3]:
df.head()
compare = df

Kolommen verwijderen en missing values opvullen

In [4]:
df = df.drop(["Height", "Weight", "ID", "Team", "Games", "Event", "Name", "City", "Sport", "NOC", "Season"], axis=1)

In [5]:
df["Medal"] = df["Medal"].fillna("None")
df['Medal'].replace(['None', 'Bronze', 'Silver', 'Gold'], [0, 1, 2, 3], inplace=True)
df['Sex'].replace(["M", "F"], [0, 1], inplace=True)
df

,Sex,Age,Year,Medal
0,0,24.0,1992,0
1,0,23.0,2012,0
2,0,24.0,1920,0
3,0,34.0,1900,3
4,1,21.0,1988,0
...,...,...,...,...
271111,0,29.0,1976,0
271112,0,27.0,2014,0
271113,0,27.0,2014,0
271114,0,30.0,1998,0


X en y waardes toekennen

In [6]:
X = df.drop('Medal', axis=1)
y = df['Medal']

X/y train/test waardes toekennen

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Data transformeren naar Dmatrix

In [8]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

D:\Anaconda3\envs\SmartSystems\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [9]:
param = {
    'eta': 0.1,
    'max_depth': 50,
    'objective': 'multi:softprob',
    'num_class': 4}
steps = 50

In [10]:
model = xgb.train(param, D_train, steps)

[11:26:53] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])
print("Precision = {}".format(precision_score(y_test, best_preds,average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy= {}".format(accuracy_score(y_test, best_preds)))

Precision = 0.29295704810804185
Recall = 0.250268214689288
Accuracy= 0.8520396872233698


In [12]:
array = []
for i in y:
    array.append(i)

In [13]:
counter = 0
for i in range(len(preds)):
    if preds[i][3] < 0.0031 and array[i] == 3:
        print('Anomaly detected: expected no medal but got', array[i], 'prediction is: ', preds[i][3])
        print(compare.loc[i])

Anomaly detected: expected no medal but got 3 prediction is:  0.0030958892
ID                         4418
Name        Alyson Regina Annan
Sex                           F
Age                        27.0
Height                    162.0
Weight                     65.0
Team                  Australia
NOC                         AUS
Games               2000 Summer
Year                       2000
Season                   Summer
City                     Sydney
Sport                    Hockey
Event     Hockey Women's Hockey
Medal                      Gold
Name: 7981, dtype: object
Anomaly detected: expected no medal but got 3 prediction is:  0.0030971337
ID                                   23355
Name                Gian Carlo Costigliolo
Sex                                      M
Age                                   26.0
Height                                 NaN
Weight                                 NaN
Team                                 Italy
NOC                                    ITA
